In [312]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import openpyxl as xl
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from sklearn.metrics import max_error
from copy import deepcopy


# Duplicates

## loading duplicates

In [ ]:
dups = pd.read_excel("matlab_XLR.xlsx", sheet_name="Duplicates")
dups.head()

In [ ]:
dups['Sample Result'] = dups['Sample Result'].str.removeprefix('<')
dups['Duplicate Result'] = dups['Duplicate Result'].str.removeprefix('<')
dups.head()

calculating range, mean, error

In [315]:
range = np.abs(dups['Sample Result'].to_numpy('float') - dups['Duplicate Result'].to_numpy('float'))
mean = np.mean(np.c_[dups['Sample Result'].to_numpy('float'), dups['Duplicate Result'].to_numpy('float')], axis=1)
error = range / mean * 100

creating new tables

In [316]:
nt = pd.DataFrame({'range': range, 'mean': mean, 'error (%)': error})

In [317]:
dups = pd.concat([dups, nt], axis=1)

# liver - muscle plot

## loading data

In [ ]:
data = pd.read_excel("matlab_XLR.xlsx", sheet_name="Results Summary")
vn = data.columns
lt_track = np.zeros(data.shape, dtype = "bool")
for i in np.arange(3, len(vn)):
    lt_track[:, i] = ~(data[vn[i]].str.removeprefix("<") == data[vn[i]])
    temp = data[vn[i]].str.removeprefix("<")
    data[vn[i]] = temp
    
data.head()


# Ploting mentioned elements

### 13 added

In [ ]:
# elements = ["Aluminum", "Zinc", "Mercury"]
## to plot all elements uncomment bellow
elements = data["Analyte"][1::2].to_list()

# to save the picture of an element figure, set the corrosponding value to True
# save_pic = [False, False, True]

## to save all uncomment below
save_pic = np.ones(len(elements), dtype="bool")


x = data.columns[3:]
# seperating liver and mt using columns contatins the word "Liver"
Liver_id = x.str.contains("Liver")
id_13 = x.str.contains("#13")
colorl = deepcopy(x.to_numpy)()
colorl[Liver_id] = 'gray'
colorl[~Liver_id] = 'darksalmon'
colorl[id_13] = 'seagreen'
ticklabel = x.str.removeprefix('White Sturgeon (WS #')
ticklabel = ticklabel.str.replace(')','')
ticklabel = ticklabel.str.replace('MT','', regex = False) 
ticklabel = ticklabel.str.replace('Liver','', regex= False) 
for i in np.arange(len(elements)):
    # creating figure
    fig, ax = plt.subplots(figsize=(10, 8))
    fig.set_facecolor("white")
    # obtaining related data
    values = np.array(data[np.logical_and(data["Analyte"] == elements[i], data["Units"] == "mg/kg wwt")].to_numpy()[:, 3:].ravel(), dtype=float)
    # bar ploting mt
    ax.bar(x[(~Liver_id)], values[(~Liver_id)], color = colorl[~Liver_id],label="Muscle")
    # bar ploting liver
    ax.bar(x[Liver_id & (~id_13)], values[Liver_id & (~id_13)], color = colorl[Liver_id & (~id_13)], label="Liver")
    ax.bar(x[Liver_id & id_13], values[Liver_id & id_13], color = colorl[Liver_id & id_13], label="juvenile by catch")
    # ploting mean of mt
    ax.plot(x[(~Liver_id) & (~id_13)], np.mean(values[(~Liver_id) & (~id_13)]) * np.ones(x[(~Liver_id) & (~id_13)].shape), color = "darkred", label = "Muscle Average", linestyle = "dashed", linewidth = 3)
    # ploting mean of liver
    ax.plot(x[Liver_id & (~id_13)], np.mean(values[Liver_id & (~id_13)]) * np.ones(x[Liver_id & (~id_13)].shape), color = "dimgray", label = "Liver Average", linestyle = "dashed", linewidth = 3)
    # setting axis side names
    ax.set_xlabel("")
    ax.set_ylabel("mg/kg wet weight", fontsize = 25)
    ax.set_title(elements[i], loc = 'left', fontsize = 30)
    # setting xticks and xtick labels
    ax.xaxis.set_ticks(np.arange(data.shape[1]-3))
    ax.xaxis.set_ticklabels(ticklabel, fontsize = 12, fontweight = "bold")

    # enabling legend
    ax.legend(fontsize = 15)
    plt.yticks(fontsize =20);
    if not os.path.exists("./pics/plot+13/"):
        os.makedirs("./pics/plot+13/")
    if save_pic[i] == True:
        fig.savefig("./pics/plot+13/"+elements[i]+".png")

# Statistics

In [320]:
from scipy.stats import shapiro
x = data.columns[3:]

## Shapiro test

In [ ]:
shapiro_dict = {}
# elements = ["Aluminum", "Zinc", "Mercury"]
elements = data["Analyte"][1::2].to_list()
for element in elements:
    result_liver = shapiro(data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt")].iloc[:, 3:].to_numpy("double")[:, Liver_id])
    result_mt = shapiro(data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt")].iloc[:, 3:].to_numpy("double")[:, ~Liver_id])
    shapiro_dict.update({element: [*result_liver, *result_mt]})

In [322]:
shapiro_table = pd.DataFrame(np.c_[np.array(list(shapiro_dict.keys()), dtype="str"), np.array(list(shapiro_dict.values()))], columns=["analyte", "liver statistic", "liver p-value", "mt statistics", "mt p-value"])

## pearson correlation factor

In [323]:
## extracting matching columns
x = data.columns[3:]
liver_temp = x[Liver_id].str.removesuffix(" Liver")
mt_temp = x[~Liver_id].str.removesuffix(" MT")
test_columns = []
for cl in liver_temp:
    if cl in mt_temp:
        test_columns.append(cl)

liver_columns = [x+" Liver" for x in test_columns]
mt_columns = [x+" MT" for x in test_columns]

In [324]:
from scipy.stats import pearsonr

In [ ]:
pearson_dict = {}
# elements = ["Aluminum", "Zinc", "Mercury"]
elements = data["Analyte"][1::2].to_list()
for element in elements:
    result = pearsonr(data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"), liver_columns].to_numpy("double").ravel(), data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"), mt_columns].to_numpy("double").ravel())
    pearson_dict.update({element: [*result]})

In [326]:
pearson_dict
pearson_liver_mt_table = pd.DataFrame(np.c_[np.array(list(pearson_dict.keys()), dtype="str")[:, np.newaxis], data.loc[data["Units"] == "mg/kg wwt", [*liver_columns, *mt_columns]].to_numpy("single"), np.array(list(pearson_dict.values()))], columns=["analyte", *[*liver_columns, *mt_columns], "statistic", "p-value"])

## pearson element by element correlation check

### Overall

In [327]:
lt_id = lt_track[:, 3:]
id_13 = x.str.contains("#13")
## to consider #13 as well uncomment next line
# id_13 = np.zeros_like(id_13, dtype="bool")

row_id = np.logical_and(data["Units"].to_numpy() == "mg/kg wwt", ~np.all(lt_id == True, axis = 1))
corr_matrix = np.corrcoef(data.loc[row_id, x[~id_13]].to_numpy("single").T, rowvar=False).round(decimals = 2)
elements = data["Analyte"].loc[row_id].to_numpy()

In [ ]:
fig, ax = plt.subplots(figsize = (34, 34))
cmap = plt.cm.PiYG
im = ax.imshow(np.tril(corr_matrix, k=0), cmap=cmap)
fig.set_facecolor("white")
im.set_clim(-1, 1)
ax.grid(False)
ax.xaxis.set_ticks(np.arange(corr_matrix.shape[0]), elements, rotation = 90, color = "r", fontsize = 20)
ax.yaxis.set_ticks(np.arange(corr_matrix.shape[0]), elements, color = "r", fontsize = 20)
# ax.set_ylim(2.5, -0.5)
for i in np.arange(corr_matrix.shape[0]):
    ax.text(i, i, elements[i], ha = 'center', va = 'center',
            color = 'white', fontsize = 9, rotation = 45, fontweight = "bold")
    for j in np.arange(i):
        ax.text(j, i, corr_matrix[i, j], ha='center', va='center',
                color='k', fontsize = 12, fontweight = "bold")
cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
cbar.ax.tick_params(labelsize=20)
plt.savefig("./pics/pearson.png")
plt.show()


### just liver

In [329]:
lt_id = lt_track[:, 3:]
row_id = np.logical_and(data["Units"].to_numpy() == "mg/kg wwt", ~np.all(lt_id[:, Liver_id] == True, axis = 1))
corr_matrix_liver = np.corrcoef(data.loc[row_id, x[Liver_id & (~id_13)]].to_numpy("single").T, rowvar=False).round(decimals = 2)
elements = data["Analyte"].loc[row_id].to_numpy()

In [ ]:
fig, ax = plt.subplots(figsize = (32, 32))
cmap = plt.cm.PiYG
im = ax.imshow(np.tril(corr_matrix_liver, k=0), cmap=cmap)
fig.set_facecolor("white")
im.set_clim(-1, 1)
ax.grid(False)
ax.xaxis.set_ticks(np.arange(corr_matrix_liver.shape[0]), elements, rotation = 90, color = "r", fontsize = 20)
ax.yaxis.set_ticks(np.arange(corr_matrix_liver.shape[0]), elements, color = "r", fontsize = 20)
# ax.set_ylim(2.5, -0.5)
for i in np.arange(corr_matrix_liver.shape[0]):
    ax.text(i, i, elements[i], ha = 'center', va = 'center',
            color = 'white', fontsize = 9, rotation = 45, fontweight = "bold")
    for j in np.arange(i):
        ax.text(j, i, corr_matrix_liver[i, j], ha='center', va='center',
                color='k', fontsize = 12, fontweight = "bold")
cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
cbar.ax.tick_params(labelsize=20)
plt.savefig("./pics/pearson liver.png")
plt.show()


### Just MT

In [331]:
lt_id = lt_track[:, 3:]
row_id = np.logical_and(data["Units"].to_numpy() == "mg/kg wwt", ~np.all(lt_id[:, ~Liver_id] == True, axis = 1))
corr_matrix_MT = np.corrcoef(data.loc[row_id, x[~Liver_id & (~id_13)]].to_numpy("single").T, rowvar=False).round(decimals = 2)
elements = data["Analyte"].loc[row_id].to_numpy()

In [ ]:
fig, ax = plt.subplots(figsize = (32, 32))
cmap = plt.cm.PiYG
im = ax.imshow(np.tril(corr_matrix_MT, k=0), cmap=cmap)
fig.set_facecolor("white")
im.set_clim(-1, 1)
ax.grid(False)
ax.xaxis.set_ticks(np.arange(corr_matrix_MT.shape[0]), elements, rotation = 90, color = "r", fontsize = 20)
ax.yaxis.set_ticks(np.arange(corr_matrix_MT.shape[0]), elements, color = "r", fontsize = 20)
# ax.set_ylim(2.5, -0.5)
for i in np.arange(corr_matrix_MT.shape[0]):
    ax.text(i, i, elements[i], ha = 'center', va = 'center',
            color = 'white', fontsize = 9, rotation = 45, fontweight = "bold")
    for j in np.arange(i):
        ax.text(j, i, corr_matrix_MT[i, j], ha='center', va='center',
                color='k', fontsize = 12, fontweight = "bold")
cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
cbar.ax.tick_params(labelsize=20)
plt.savefig("./pics/pearson mt.png")
plt.show()


### corolation for selected elements
MT first

In [333]:
lt_id = lt_track[:, 3:]
elements = ["Lead", "Cadmium", "Molybdenum", "Mercury", "Selenium", "Copper", "Zinc"]
element_id = np.zeros(data["Analyte"].shape, dtype="bool")
for i in np.arange(element_id.shape[0]):
    if data["Analyte"][i] in elements:
        element_id[i] = True
row_id = np.logical_and(data["Units"].to_numpy() == "mg/kg wwt", ~np.all(lt_id[:, ~Liver_id] == True, axis = 1)) * element_id
corr_matrix_MT = np.corrcoef(data.loc[row_id, x[~Liver_id & (~id_13)]].to_numpy("single").T, rowvar=False).round(decimals = 2)
elements = data["Analyte"].loc[row_id].to_numpy()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 15))
cmap = plt.cm.PiYG
im = ax.imshow(np.tril(corr_matrix_MT, k=0), cmap=cmap)
fig.set_facecolor("white")
im.set_clim(-1, 1)
ax.grid(False)
ax.xaxis.set_ticks(np.arange(corr_matrix_MT.shape[0]), elements, rotation = 90, color = "r", fontsize = 20)
ax.yaxis.set_ticks(np.arange(corr_matrix_MT.shape[0]), elements, color = "r", fontsize = 20)
# ax.set_ylim(2.5, -0.5)
for i in np.arange(corr_matrix_MT.shape[0]):
    ax.text(i, i, elements[i], ha = 'center', va = 'center',
            color = 'white', fontsize = 12, rotation = 45, fontweight = "bold")
    for j in np.arange(i):
        ax.text(j, i, corr_matrix_MT[i, j], ha='center', va='center',
                color='k', fontsize = 20, fontweight = "bold")
cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
plt.savefig("./pics/pearson_selected elements_MT.png")
plt.show()


Liver

In [ ]:
lt_id = lt_track[:, 3:]
elements = ["Lead", "Cadmium", "Molybdenum", "Mercury", "Selenium", "Copper", "Zinc"]
element_id = np.zeros(data["Analyte"].shape, dtype="bool")
for i in np.arange(element_id.shape[0]):
    if data["Analyte"][i] in elements:
        element_id[i] = True
row_id = np.logical_and(data["Units"].to_numpy() == "mg/kg wwt", ~np.all(lt_id[:, Liver_id] == True, axis = 1)) * element_id
corr_matrix_MT = np.corrcoef(data.loc[row_id, x[Liver_id & (~id_13)]].to_numpy("single").T, rowvar=False).round(decimals = 2)
elements = data["Analyte"].loc[row_id].to_numpy()

fig, ax = plt.subplots(figsize = (15, 15))
cmap = plt.cm.PiYG
im = ax.imshow(np.tril(corr_matrix_MT, k=0), cmap=cmap)
fig.set_facecolor("white")
im.set_clim(-1, 1)
ax.grid(False)
ax.xaxis.set_ticks(np.arange(corr_matrix_MT.shape[0]), elements, rotation = 90, color = "r", fontsize = 20)
ax.yaxis.set_ticks(np.arange(corr_matrix_MT.shape[0]), elements, color = "r", fontsize = 20)
# ax.set_ylim(2.5, -0.5)
for i in np.arange(corr_matrix_MT.shape[0]):
    ax.text(i, i, elements[i], ha = 'center', va = 'center',
            color = 'white', fontsize = 12, rotation = 45, fontweight = "bold")
    for j in np.arange(i):
        ax.text(j, i, corr_matrix_MT[i, j], ha='center', va='center',
                color='k', fontsize = 20, fontweight = "bold")
cbar = ax.figure.colorbar(im, ax=ax, format='% .2f')
cbar.ax.tick_params(labelsize=20)
plt.savefig("./pics/pearson_selected elements_Liver.png")
plt.show()

# One way Anova

In [ ]:
from scipy.stats import f_oneway

anova_dict = {}
# elements = ["Aluminum", "Zinc", "Mercury"]
elements = data["Analyte"][1::2].to_list()
for element in elements:
    result = f_oneway(data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"),
                                liver_columns].to_numpy("double").ravel(), 
                                data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"), 
                                         mt_columns].to_numpy("double").ravel())
    anova_dict.update({element: [*result]})

anova_table = pd.DataFrame(np.c_[np.array(list(anova_dict.keys()), dtype="str")[:, np.newaxis],
                                  data.loc[data["Units"] == "mg/kg wwt",
                                            [*liver_columns, *mt_columns]].to_numpy("single"),
                                              np.array(list(anova_dict.values()))],
                                                columns=["analyte", *[*liver_columns, *mt_columns],
                                                          "statistic", "p-value"])

## T test

In [ ]:
from scipy.stats import ttest_ind

ttest_dict = {}
# elements = ["Aluminum", "Zinc", "Mercury"]
elements = data["Analyte"][1::2].to_list()
for element in elements:
    result = ttest_ind(data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"),
                                liver_columns].to_numpy("double").ravel(), 
                                data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"), 
                                         mt_columns].to_numpy("double").ravel())
    ttest_dict.update({element: [*result]})

ttest_table = pd.DataFrame(np.c_[np.array(list(ttest_dict.keys()), dtype="str")[:, np.newaxis],
                                  data.loc[data["Units"] == "mg/kg wwt",
                                            [*liver_columns, *mt_columns]].to_numpy("single"),
                                              np.array(list(ttest_dict.values()))],
                                                columns=["analyte", *[*liver_columns, *mt_columns],
                                                          "statistic", "p-value"])

In [338]:
from scipy.stats import kruskal

kruskal_dict = {}
elements = ["Cadmium", "Lead", "Selenium"]
# elements = data["Analyte"][1::2].to_list()
element_id = np.array([x in elements for x in data['Analyte']], dtype='bool')[1::2]
# not_happend = []
for element in elements:
    try:
      gp1 = data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"),
                                  liver_columns].to_numpy("double").ravel()
      gp2 = data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"), 
                                          mt_columns].to_numpy("double").ravel()
      result, p_value = kruskal(gp1, gp2)
      kruskal_dict.update({element: [result, p_value]})
    except ValueError:
       kruskal_dict.update({element: [np.NaN, np.NaN]})
       

kruskal_table = pd.DataFrame(np.c_[np.array(list(kruskal_dict.keys()), dtype="str")[:, np.newaxis],
                                  data.loc[data["Units"] == "mg/kg wwt",
                                            [*liver_columns, *mt_columns]].to_numpy("single")[element_id, :],
                                              np.array(list(kruskal_dict.values()))],
                                                columns=["analyte", *[*liver_columns, *mt_columns],
                                                          "h", "p-value"])

## Mannwhitney U Test

In [339]:
from scipy.stats import mannwhitneyu

mannwhit = {}
# elements = ["Cadmium", "Lead", "Selenium"]
elements = data["Analyte"][1::2].to_list()
element_id = np.array([x in elements for x in data['Analyte']], dtype='bool')[1::2]
for element in elements:
    try:
      gp1 = data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"),
                                  liver_columns].to_numpy("double").ravel()
      gp2 = data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt"), 
                                          mt_columns].to_numpy("double").ravel()
      result, p_value = mannwhitneyu(gp1, gp2)
      mannwhit.update({element: [result, p_value]})
    except ValueError:
       mannwhit.update({element: [np.NaN, np.NaN]})
       

mannwhit_table = pd.DataFrame(np.c_[np.array(list(mannwhit.keys()), dtype="str")[:, np.newaxis],
                                  data.loc[data["Units"] == "mg/kg wwt",
                                            [*liver_columns, *mt_columns]].to_numpy("single")[element_id, :],
                                              np.array(list(mannwhit.values()))],
                                                columns=["analyte", *[*liver_columns, *mt_columns],
                                                          "h", "p-value"])

## Outliers

In [340]:
outlier_dict = {}
# elements = ["Aluminum"]
elements = data["Analyte"][1::2].to_list()
for element in elements:
    temp = data.loc[np.logical_and(data["Analyte"] == element, data["Units"] == "mg/kg wwt")].iloc[:, 3:].to_numpy("double")
    q1 = np.percentile(temp[:, Liver_id], 25)
    q3 = np.percentile(temp[:, Liver_id], 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    cols = data.columns[3:][Liver_id]
    outlier_liver = cols[((temp[:, Liver_id] < lower_bound) | (temp[:, Liver_id] > upper_bound)).ravel()]
    q1 = np.percentile(temp[:, ~Liver_id], 25)
    q3 = np.percentile(temp[:, ~Liver_id], 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    cols = data.columns[3:][~Liver_id]
    outlier_muscle = cols[((temp[:, ~Liver_id] < lower_bound) | (temp[:, ~Liver_id] > upper_bound)).ravel()]
    
    outlier_dict.update({element: [", ".join(outlier_liver), ", ".join(outlier_muscle)]})
outlier_table = pd.DataFrame(np.c_[np.array(list(outlier_dict.keys()), dtype="str"), np.array(list(outlier_dict.values()))], columns=["analyte", "liver outlier", "muscle outlier"])

# Reading Punch Data

In [341]:
punch_data = pd.read_excel("2303384 FINAL BAL Standard EDD 18 May 23 1649.xls")

temp = punch_data["Sample_Tag"].str.split("- ")
tag = []
tissue = []
for i in np.arange(temp.shape[0]):
    if len(temp[i]) > 1:
        tag.append(temp[i][0].strip())
        tissue.append(temp[i][1].strip())
    else:
        tag.append((temp[i][0]).strip())
        tissue.append("None")

punch_data.insert(4, "Tag", tag)
punch_data.insert(4, "Tissue", tissue)
punch_data.drop(columns="Sample_Tag")

# reading [fish_number - tag] from sheet excel sheet names
temp_file = xl.reader.excel.load_workbook("2022 Nechako river WSG Mortality Summary.xlsx")
tags = temp_file.sheetnames
tags = np.char.split(tags, "-")
fish_number = []
fish_tag = []
for tag in tags:
    fish_number.append(tag[0].strip())
    fish_tag.append(tag[1].strip())
tag_list = np.array(np.c_[fish_number, fish_tag])

# maping tags to fish_numbers
temp = punch_data['Tag'].copy().to_numpy()
for i in np.arange(tag_list.shape[0]):
    temp[punch_data['Tag'] == tag_list[i, 1]] = tag_list[i, 0]

# replacing the results with Sample_Tag column :)
punch_data["Sample_Tag"] = temp

punch_data = punch_data.loc[(punch_data["Tissue"] == "Liver") | (punch_data["Tissue"] == "Muscle")]
# print(punch_data.columns)
punch_data = punch_data.loc[:, punch_data.columns[[4, 5, 6, 11, 12, 14, 15, 28]]]

element_ref = np.c_[np.unique(data["Analyte"].to_numpy()[1:]), ["Al", "An", "As", "Ba", "Be", "Bi", "B", "Cd", "Ca", "Ce", "Cr", "Co", "Cu", "Fn", "Pb", "Li", "Mg", "Mn", "Hg", "Mo", "Ni", "P", "K", "Ru", "Se", "Na", "St", "Te", "Ta", "Tin", "Ur", "Va", "Zn", "Zi"]]

elements = [element_ref[i==element_ref[:, 1], 0].item() for i in punch_data["Analyte"]]
punch_data["Analyte"] = elements

punch_data.replace("Muscle", "MT", inplace=True)
punch_data.reset_index(inplace=True)

ours = punch_data["Result"].to_numpy("single").copy()
for i in np.arange(ours.shape[0]):
    temp = data.loc[(data["Analyte"] == punch_data["Analyte"][i]) & (data["Units"] == "mg/kg wwt"), data.columns.str.contains(punch_data["Tissue"][i]) & data.columns.str.contains(punch_data["Sample_Tag"][i])].to_numpy("single")
    if temp.shape[1] == 0:
        ours[i] = None
    else:
        ours[i] = temp.item(0)

punch_data.insert(5, column="Ours", value=ours)

In [ ]:
punch_data

In [ ]:
shapiro_table

# Reading Punch Data

In [344]:
punch_data = pd.read_excel("2303384 FINAL BAL Standard EDD 18 May 23 1649.xls")

temp = punch_data["Sample_Tag"].str.split("- ")
tag = []
tissue = []
for i in np.arange(temp.shape[0]):
    if len(temp[i]) > 1:
        tag.append(temp[i][0].strip())
        tissue.append(temp[i][1].strip())
    else:
        tag.append((temp[i][0]).strip())
        tissue.append("None")

punch_data.insert(4, "Tag", tag)
punch_data.insert(4, "Tissue", tissue)
punch_data.drop(columns="Sample_Tag")

# reading [fish_number - tag] from sheet excel sheet names
temp_file = xl.reader.excel.load_workbook("2022 Nechako river WSG Mortality Summary.xlsx")
tags = temp_file.sheetnames
tags = np.char.split(tags, "-")
fish_number = []
fish_tag = []
for tag in tags:
    fish_number.append(tag[0].strip())
    fish_tag.append(tag[1].strip())
tag_list = np.array(np.c_[fish_number, fish_tag])

# maping tags to fish_numbers
temp = punch_data['Tag'].copy().to_numpy()
for i in np.arange(tag_list.shape[0]):
    temp[punch_data['Tag'] == tag_list[i, 1]] = tag_list[i, 0]

# replacing the results with Sample_Tag column :)
punch_data["Sample_Tag"] = temp

punch_data = punch_data.loc[(punch_data["Tissue"] == "Liver") | (punch_data["Tissue"] == "Muscle")]
# print(punch_data.columns)
punch_data = punch_data.loc[:, punch_data.columns[[4, 5, 6, 11, 12, 14, 15, 28]]]

element_ref = np.c_[np.unique(data["Analyte"].to_numpy()[1:]), ["Al", "An", "As", "Ba", "Be", "Bi", "B", "Cd", "Ca", "Ce", "Cr", "Co", "Cu", "Fn", "Pb", "Li", "Mg", "Mn", "Hg", "Mo", "Ni", "P", "K", "Ru", "Se", "Na", "St", "Te", "Ta", "Tin", "Ur", "Va", "Zn", "Zi"]]

elements = [element_ref[i==element_ref[:, 1], 0].item() for i in punch_data["Analyte"]]
punch_data["Analyte"] = elements

punch_data.replace("Muscle", "MT", inplace=True)
punch_data.reset_index(inplace=True)

ours = punch_data["Result"].to_numpy("single").copy()
for i in np.arange(ours.shape[0]):
    temp = data.loc[(data["Analyte"] == punch_data["Analyte"][i]) & (data["Units"] == "mg/kg wwt"), data.columns.str.contains(punch_data["Tissue"][i]) & data.columns.str.contains(punch_data["Sample_Tag"][i])].to_numpy("single")
    if temp.shape[1] == 0:
        ours[i] = None
    else:
        ours[i] = temp.item(0)

punch_data.insert(5, column="Ours", value=ours)

## Ploting punch data vs Ours

In [ ]:
elements = punch_data["Analyte"].unique()
# elements = ["Zinc"]
tissues = punch_data["Tissue"].unique()
tissue_names = tissues.copy()
tissue_names[tissue_names == 'MT'] = "Muscle"
for element in elements:
    for tissue, name in zip(tissues, tissue_names):
        fig, ax = plt.subplots(figsize = (10, 8))
        fig.set_facecolor("white")
        temp = punch_data[["Sample_Tag", "Result", "Ours"]].loc[(punch_data["Tissue"] == tissue) & (punch_data["Analyte"] == element) & (punch_data["Matrix"] == "Fish")].dropna()
        temp = temp.reindex(temp["Sample_Tag"].str.removeprefix("#").astype("int").sort_values().index)
        x = np.arange(temp["Sample_Tag"].shape[0])
        ax.bar(x, temp["Result"].to_numpy("single").T, width = 0.4, label = "Punch", color = "royalblue")
        ax.bar(x+.4, temp["Ours"].to_numpy("single").T, width = 0.4, label = name, color = "lightcoral") 
        ax.set_xticks(x+0.2, temp["Sample_Tag"].str.removeprefix("#"), fontsize = 15, fontweight = "bold")
        plt.yticks(fontsize = 15)
        plt.xticks(fontsize = 15)

        ax.set_title(f"{element}", fontsize = 25, loc = "left") #in {name}"
        ax.set_xlabel("Fish ID", fontsize = 20)
        ax.set_ylabel("mg/kg wet weight",fontsize = 20)
        ax.legend(fontsize = 20)
        if not os.path.exists("./pics/punch_ours/"):
            os.makedirs("./pics/punch_ours/")
        fig.savefig(f"./pics/punch_ours/{element}_{tissue}.png")

## Muscle punch vs Our Liver

## punch vs Ours correlation

In [346]:
mus_po_cor_dict = {}
# elements = ["Aluminum", "Zinc", "Mercury"]
elements = punch_data["Analyte"].unique()
for element in elements:
    row_id = np.logical_and(punch_data["Analyte"] == element, punch_data["Matrix"] == "Fish") & (punch_data["Tissue"] == "MT")
    temp = punch_data.loc[row_id, ["Sample_Tag" ,"Result", "Ours"]].dropna()
    result = pearsonr(temp["Ours"].to_numpy("double").ravel(),
                       temp["Result"].to_numpy("double").ravel())
    mus_po_cor_dict.update({element: [temp, result]})

In [347]:
liver_po_cor_dict = {}
# elements = ["Aluminum", "Zinc", "Mercury"]
elements = punch_data["Analyte"].unique()
for element in elements:
    row_id = np.logical_and(punch_data["Analyte"] == element, punch_data["Matrix"] == "Fish") & (punch_data["Tissue"] == "Liver")
    temp = punch_data.loc[row_id, ["Sample_Tag", "Ours"]].dropna()
    temp2 = punch_data.loc[np.logical_and(punch_data["Analyte"] == element, punch_data["Matrix"] == "Fish") & (punch_data["Tissue"] == "MT"), ["Sample_Tag", "Result"]]
    temp = temp.merge(temp2, 'left', on = 'Sample_Tag')
    result = pearsonr(temp["Ours"].to_numpy("double").ravel(),
                       temp["Result"].to_numpy("double").ravel())
    liver_po_cor_dict.update({element: [temp, result]})

In [348]:
tags = list(mus_po_cor_dict.values())[0][0]["Sample_Tag"].to_numpy()
values = np.zeros((len(mus_po_cor_dict), tags.shape[0] * 2), dtype="single")
stat = np.zeros((len(mus_po_cor_dict), 2))
for i in np.arange(len(mus_po_cor_dict)):
    item = list(mus_po_cor_dict.keys())[i]
    values[i, :] = np.r_[mus_po_cor_dict[item][0]["Result"].to_numpy("single"), mus_po_cor_dict[item][0]["Ours"].to_numpy("single")]
    stat[i, :] = np.array(mus_po_cor_dict[item][1]).round(2)
mus_po_cor_table = pd.DataFrame(np.c_[np.array(list(mus_po_cor_dict.keys()), dtype="str")[:, np.newaxis], values, stat],
                                                columns=["analyte", *[*(tags + " Punch"), *(tags + " Ours")],
                                                          "statistic", "p-value"])

In [349]:
tags = list(liver_po_cor_dict.values())[0][0]["Sample_Tag"].to_numpy()
values = np.zeros((len(liver_po_cor_dict), tags.shape[0] * 2), dtype="single")
stat = np.zeros((len(liver_po_cor_dict), 2))
for i in np.arange(len(liver_po_cor_dict)):
    item = list(liver_po_cor_dict.keys())[i]
    values[i, :] = np.r_[liver_po_cor_dict[item][0]["Result"].to_numpy("single"), liver_po_cor_dict[item][0]["Ours"].to_numpy("single")]
    stat[i, :] = np.array(liver_po_cor_dict[item][1]).round(2)
liver_po_cor_table = pd.DataFrame(np.c_[np.array(list(liver_po_cor_dict.keys()), dtype="str")[:, np.newaxis], values, stat],
                                                columns=["analyte", *[*(tags + " Punch"), *(tags + " Ours")],
                                                          "statistic", "p-value"])

# Fishes height and weight

### reading fishes properties and calculating Fulton's condition factor

In [ ]:
fishes = pd.read_excel("Copy of 2022 Sturgeon mortalities Biodata - Nechako River tracking Sept 22 2022.xlsx" , sheet_name='2022 mortality track')[["Tracking #", "TOTAL", "kg"]].dropna().rename(columns={"Tracking #" : "tag", "TOTAL" : "length", "kg":"weight"})
fishes.reset_index(inplace=True, drop=True)
b = 3 ## fulton factor coefficient
fulton = fishes["weight"] * 100000 / (fishes["length"] ** b)
fishes.insert(3, "fulton", fulton)
fishes

In [ ]:
x = fishes['tag'].astype('str')
y = fishes['fulton']
colorl = ['darkgray'] * (len(x)-1) + ['seagreen']
plt.figure(figsize= (15,10))
plt.bar(x,y,color = colorl)
#plt.title('Condition Factor',fontsize = 30, fontweight = 'bold',fontdict = {'family': 'serif'
       # 'color':  'darkred',
        #'weight': 'normal',
        #'size': 16,
       # })

plt.xlabel('Fish ID', fontsize = 30)
plt.ylabel('Fulton Condition Factor (K)', fontsize = 30)
plt.xticks(fontsize =20)
plt.yticks(fontsize =20);

if not os.path.exists('./pics/fulton'):
    os.mkdir('./pics/fulton')

plt.savefig('./pics/fulton/barplot.jpg')


In [352]:
# elements = ["Aluminum", "Zinc"]
elements = data["Analyte"][1::2]
tissues = ["Liver", "MT"]
fish_dict = {}
for element in elements:
    id_liver = np.empty_like(fishes.index, dtype="bool")
    id_mt = np.empty_like(fishes.index, dtype="bool")
    for i in np.arange(id_mt.shape[0]):
        id_liver[i] = np.any(data.columns == f"White Sturgeon (WS #{fishes['tag'][i]}) Liver")
        id_mt[i] = np.any((data.columns == f"White Sturgeon (WS #{fishes['tag'][i]}) MT"))
    row_id = (data["Analyte"] == element) & (data["Units"] == "mg/kg wwt")
    c_liver = np.char.add(np.char.add("White Sturgeon (WS #", fishes['tag'][id_liver].to_numpy("str")), ") Liver")
    c_MT = np.char.add(np.char.add("White Sturgeon (WS #", fishes['tag'][id_mt].to_numpy("str")), ") MT")
    temp_liver = fishes.loc[id_liver, :]
    temp_liver.insert(3, "liver", data.loc[row_id, c_liver].to_numpy().T)
    temp_mt = fishes.loc[id_mt, :]
    temp_mt.insert(3, "MT", data.loc[row_id, c_MT].to_numpy().T)
    fish_dict.update({element: [temp_liver, temp_mt]})
        

### Scatter plot

In [ ]:
for element in fish_dict:    
    liver = fish_dict[element][0]
    mt = fish_dict[element][1]
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    fig.set_facecolor("White")
    ax[0].scatter(*(liver[["length", "liver"]].to_numpy("float").T), label="Liver")
    ax[0].scatter(*(mt[["length", "MT"]].to_numpy("float").T), label="MT")
    ax[0].set_xlabel("Length (cm)")
    ax[0].set_ylabel("mg/kg wwt")
    ax[0].legend()
    ax[1].scatter(*(liver[["weight", "liver"]].to_numpy("float").T), label="Liver")
    ax[1].scatter(*(mt[["weight", "MT"]].to_numpy("float").T), label="MT")
    ax[1].set_xlabel("Weight (kg)")
    ax[1].set_ylabel("mg/kg wwt")
    ax[1].legend()
    fig.suptitle(element, fontsize=20, fontweight = "bold")
    if not os.path.exists("./pics/scatter/"):
        os.mkdir("./pics/scatter/")
    fig.savefig(f"./pics/scatter/{element}.png")

### Fulton Factor Plot

In [ ]:
for element in fish_dict:    
    liver = fish_dict[element][0]
    mt = fish_dict[element][1]
    fig, ax = plt.subplots(figsize=(8, 6))
    fig.set_facecolor("White")
    ax.scatter(*(liver[["fulton", "liver"]].to_numpy("float").T), label="Liver")
    ax.scatter(*(mt[["fulton", "MT"]].to_numpy("float").T), label="MT")
    ax.set_xlabel("Fulton's condition factor")
    ax.set_ylabel("mg/kg wwt")
    ax.legend()
    fig.suptitle(element, fontsize=20, fontweight = "bold")
    if not os.path.exists("./pics/scatter/fulton/"):
        os.mkdir("./pics/scatter/fulton/")
    fig.savefig(f"./pics/scatter/fulton/{element}.png")

## making excel for statistics
 dups, shapiro, pearson, anova, punch vs ours -> stats.xlsx

In [355]:
with pd.ExcelWriter('stats.xlsx') as writer:
    dups.to_excel(writer, sheet_name="dups")
    shapiro_table.to_excel(writer, sheet_name="shapiro")
    pearson_liver_mt_table.to_excel(writer, sheet_name="pearson liver-mt")
    anova_table.to_excel(writer, sheet_name= "One Way Anova")
    ttest_table.to_excel(writer, sheet_name = "t_test")
    mus_po_cor_table.to_excel(writer, sheet_name= "punch vs ours MT Cor")
    liver_po_cor_table.to_excel(writer, sheet_name= "punch vs ours liver Cor")
    kruskal_table.to_excel(writer, sheet_name = 'kruskal')
    mannwhit_table.to_excel(writer, sheet_name = 'mannwhitney')
    outlier_table.to_excel(writer, sheet_name = 'outlier')
    

In [ ]:
analyte_set = ["Mercury", "Selenium"]
tissues = ['mus', 'liver']
names = ['Muscle', 'Liver']
for tissue, name in zip(tissues, names):
    for analyte in analyte_set:
        dict_name = eval(tissue + "_po_cor_dict")
        temp = dict_name[analyte][0]
        temp = temp.reindex(temp["Sample_Tag"].str.removeprefix("#").astype("int8").sort_values().index)
        model = LinearRegression()
        model.fit(temp["Result"].to_numpy().reshape(-1, 1), temp["Ours"].to_numpy())
        r_sq = model.score(temp["Result"].to_numpy().reshape(-1, 1), temp["Ours"].to_numpy())
        fig, ax = plt.subplots(figsize=(8, 6))
        fig.set_facecolor("white")
        ax.scatter(*temp[["Result", "Ours"]].to_numpy().T)
        ax.set_xlabel("Punch", fontsize = 20)
        ax.set_ylabel(name, fontsize = 25)
        ax.set_title(analyte, fontsize = 25)
        ax.plot(temp["Result"].to_numpy().reshape(-1, 1), model.predict(temp["Result"].to_numpy().reshape(-1, 1)), color = "darkred", label = 'regressed')
        merror = max_error(temp["Ours"].to_numpy(), model.predict(temp["Result"].to_numpy().reshape(-1, 1)))
        ax.annotate(f"$r^2$ = {r_sq.round(2)}\n{name} = {model.coef_[0].round(2)} * punch + {model.intercept_.round(2)}\nmax_error = {merror:.2f}",
                    (0.01*np.diff(ax.get_xlim())[0]+ax.get_xlim()[0], ax.get_ylim()[1]-0.23*np.diff(ax.get_ylim())[0]), color = "k", fontweight = "bold", fontsize = 15);
        ax.plot(temp["Result"].to_numpy().reshape(-1, 1), temp["Result"].to_numpy().reshape(-1, 1), label = "1:1")
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plt.legend(loc = 'lower right')
        if not os.path.exists("./pics/Reg"):
            os.mkdir("./pics/Reg")
        if not os.path.exists(f"./pics/Reg/{name}"):
            os.mkdir(f"./pics/Reg/{name}")
        fig.savefig(f"./pics/Reg/{name}/{analyte}.png")

## Hatchery data

In [357]:
# bismuth, cesium, Lithium, Phosphorus, Rubidum, Strontium, Tellurium and Zirconium are the 8 elements that exist in our data but not in hatcheri data.
# inorganic arsenic, Silicon, Silver and Sulfur are the 4 elements that exist in Hatcheri data but not in our data.
hatcheri_A = pd.read_excel("hatcheri.xlsx", sheet_name= "A")
hatcheri_B = pd.read_excel("Hatcheri.xlsx", sheet_name= "B")
hatcheri_C = pd.read_excel("Hatcheri.xlsx", sheet_name= "C")

In [358]:
elements = np.array([e for e in data.Analyte[1::2].to_numpy() if any(hatcheri_A.Analyte.str.contains(e))])
mins = np.empty_like(elements, dtype="float")
maxs = mins.copy()
means = mins.copy()
sd = mins.copy()
counts = mins.copy()
detected = mins.copy()

In [359]:
for i in np.arange(elements.shape[0]):
    temp = data.loc[np.logical_and(data["Analyte"] == elements[i], data["Units"] == "mg/kg wwt"), data.columns.str.contains("MT")].to_numpy("float")
    dl = data.loc[np.logical_and(data["Analyte"] == elements[i], data["Units"] == "mg/kg wwt"), "Lowest\nDetection Limit"].to_numpy("float")
    mins[i] = temp.min()
    means[i] = temp.mean()
    maxs[i] = temp.max()
    sd[i] = temp.std()
    counts[i] = temp.shape[1]
    detected[i] = (1-(np.sum(temp == dl)/temp.shape[1])) * 100


In [360]:
hatcheri_D = pd.DataFrame({'Analyte': elements, 'Count of Results': counts.astype(np.int8), 'Detected':detected, 'Min': mins, 'Mean': means, 'Max': maxs, 'SD': sd})

In [361]:
import pandas as pd
from openpyxl import load_workbook

excel_path = 'Hatcheri.xlsx'
book = load_workbook(excel_path)
writer = pd.ExcelWriter(excel_path, engine='openpyxl') 
if 'D' in book.sheetnames:
    del book['D']
writer.book = book
hatcheri_D.to_excel(writer, index=False, sheet_name="D")

writer.save()
writer.close()


In [362]:
hatcheri_A["Size"] = "A"
hatcheri_B["Size"] = "B"
hatcheri_C["Size"] = "C"
hatcheri_D["Size"] = "D"

In [ ]:
hatcheri = hatcheri_A.append([hatcheri_B, hatcheri_C, hatcheri_D])

In [364]:
hatcheri.sort_values(by="Analyte", inplace=True)

In [ ]:
hatcheri

In [366]:
elements = hatcheri["Analyte"].unique()

In [ ]:
for element in elements:
    plt.figure(facecolor="white", figsize=(10, 10))
    plt.title(element)
    temp = hatcheri.loc[hatcheri.Analyte == element, :].drop(columns=["Analyte"])
    temp.sort_values(by="Size", inplace=True)
    x = temp.Size
    mins = temp.Min
    maxs = temp.Max
    means = temp.Mean
    sd = temp.SD
    plt.errorbar(x, means, sd, fmt='ok', lw=20, label = "std")
    plt.errorbar(x, means, [means - mins, maxs - means],
             fmt='o', color='gray', ecolor='gray', lw=3, label = "fillet")
    plt.legend()
    plt.xlabel("Size")
    plt.ylabel("mg/kg wwt")
    if not os.path.exists("./pics/Hatcheri"):
        os.mkdir("./pics/Hatcheri")
    plt.savefig(f"./pics/Hatcheri/{element}.png")
    

# PCB Finclip

In [ ]:
col_names = pd.read_excel("PCB.xls", sheet_name="DATA", nrows=0).columns
col_names = np.array(col_names[~col_names.str.contains("Unnamed")][1:].repeat(3))
for i in np.arange(tag_list.shape[0]):
    col_names[col_names == tag_list[i, 1]] = tag_list[i, 0][1:]
pcb = pd.read_excel("PCB.xls", sheet_name="DATA", skiprows = 5)
pcb = pcb.loc[:int(np.where(pcb['UNITS'] == r"% Lipid")[0]), :]
suffix = ['flag', 'value', 'RL'] * int(len(col_names) / 3)
cols = [i + '_' + j for i, j in zip(col_names, suffix)]
cols.insert(0, 'Units')
pcb.columns = cols
pcb.set_index('Units', inplace=True)

blank_columns = np.where(pcb.columns.str.contains('Blank'))[0]
rl_nd = pcb.loc[pcb.iloc[:, blank_columns[0]].str.contains(r'\bND\b(?!R)', na = False).to_numpy(), pcb.columns[blank_columns[2]]].dropna()
pcb
# Unit: pg/g (wet weight basis)

### Blank correction

In [ ]:
pcb_corrected = pcb.copy().iloc[:, 1:-2:3]
pcb_corrected.columns = pcb_corrected.columns.str.removesuffix('_value')
pcb_corrected.loc[rl_nd.index, pcb_corrected.columns[-1]] = rl_nd.to_numpy() / np.sqrt(2)
id = pcb_corrected.iloc[:-1, -1].dropna().index
pcb_corrected.loc[id, pcb_corrected.columns[:-1]] -= pcb_corrected.loc[id, pcb_corrected.columns[-1]].to_numpy()[:, np.newaxis]
pcb_corrected = pcb_corrected.iloc[:, :-1]
print(f'count of samples corrected to negative: {np.sum((~(pcb_corrected.isna() | (pcb_corrected > 0)).to_numpy()))}')
print(f'position of corrected samples: {np.where((~(pcb_corrected.isna() | (pcb_corrected > 0))).to_numpy())}')
pcb_corrected.iloc[np.where((~(pcb_corrected.isna() | (pcb_corrected > 0))).to_numpy())] = 0


### Lipid Normalization

In [370]:
lipid_normalized = pcb_corrected.copy()
lipid_normalized.iloc[-1, :] /= 100
lipid_normalized.iloc[:-1, :] /= lipid_normalized.iloc[-1, :]
lipid_normalized.index = list(lipid_normalized.index[:-1])+['Lipid']

In [371]:
if not os.path.exists("./generated excels/"):
    os.mkdir("./generated excels/")
with pd.ExcelWriter('./generated excels/pcb.xlsx') as writer:
    pcb_corrected.to_excel(writer, sheet_name="blank_corrected")
    lipid_normalized.to_excel(writer, sheet_name="lipid_normalized")

In [372]:
total_pcbs = pcb_corrected.loc['TOTAL PCBs', :].dropna()

In [ ]:
total_pcbs = pcb_corrected.loc['TOTAL PCBs', :].dropna()
refs = pd.DataFrame(np.array([0.036, 0.0424, 2.5, .7, 2.5]) * 1e3, index = ["2009 RBC", 'Updated RBC', 'MRL', 'MDL', 'ACG'])
n = total_pcbs.shape[0] + refs.shape[0]
fig, ax = plt.subplots(figsize=(15, 15), facecolor = "white")
ax.bar(total_pcbs.index[:-1], total_pcbs.to_numpy()[:-1], color = 'c', label = "Nechako Mass Mortality")
ax.bar(total_pcbs.index[-1], total_pcbs.to_numpy()[-1], color = 'seagreen', label = "Juvenile by Catch")
ax.bar(refs.index, refs.loc[:, 0].to_numpy(), color = 'k', label = 'Literature')
ax.xaxis.set_ticks(np.arange(n))
ax.xaxis.set_ticklabels(list(total_pcbs.index.str.removesuffix("_value").str.removesuffix('-Trembleur')) + list(refs.index), rotation = 30, ha = 'right', fontsize = 12, fontweight = 'bold');
ax.set_title("TOTAL PCBs")
ax.set_ylabel("pg/g (wet weight basis)", fontsize = 15, fontweight = 'bold')
ax.set_xlabel("Fish ID", fontsize = 12, fontweight = 'bold')
plt.legend();
if not os.path.exists("./pics/PCBs"):
    os.mkdir("./pics/PCBs")
plt.savefig("./pics/PCBs/Total_PCBs.png")

In [ ]:

n = total_pcbs.shape[0] 
fig, ax = plt.subplots(figsize=(6, 4), facecolor = "white")
ax.bar(total_pcbs.index, total_pcbs.to_numpy(), color = 'g', label = "Ours")

ax.xaxis.set_ticks(np.arange(n))
ax.xaxis.set_ticklabels(list(total_pcbs.index.str.removesuffix("_value").str.removesuffix('-Trembleur')), fontsize = 12, fontweight = 'bold');
ax.set_title("TOTAL PCBs")
ax.set_ylabel("pg/g (wet weight basis)", fontsize = 15, fontweight = 'bold')
ax.set_xlabel("Fish ID", fontsize = 12, fontweight = 'bold')
plt.legend();
if not os.path.exists("./pics/PCBs"):
    os.mkdir("./pics/PCBs")
plt.savefig("./pics/PCBs/Total_PCBsNoref.png")

In [ ]:
total_pcbs = lipid_normalized.loc['TOTAL PCBs', :].dropna()/1000000
n = total_pcbs.shape[0] 
fig, ax = plt.subplots(figsize=(6, 4), facecolor = "white")
ax.bar(total_pcbs.index[:-1], total_pcbs.to_numpy()[:-1], color = 'k', label = "Ours")
ax.bar(total_pcbs.index[-1], total_pcbs.to_numpy()[-1], color = 'seagreen', label = "Ours")

ax.xaxis.set_ticks(np.arange(n))
ax.xaxis.set_ticklabels(list(total_pcbs.index.str.removesuffix("_value").str.removesuffix('-Trembleur')), fontsize = 12, fontweight = 'bold');
ax.set_title("Total Polychlorinated Biphenyls (PCBs)")
ax.set_ylabel("$\Sigma PCB \enspace mg/kg \enspace lw$", fontsize = 15, fontweight = 'bold')
ax.set_xlabel("Fish ID", fontsize = 12, fontweight = 'bold')
#plt.legend();
if not os.path.exists("./pics/PCBs"):
    os.mkdir("./pics/PCBs")
plt.savefig("./pics/PCBs/Total_PCBsLipidN.png")

### Pie Chart

In [ ]:
fig, ax = plt.subplots(1, pcb_corrected.shape[1], figsize = (13.39, 3))
fig.set_facecolor("white")
fig.suptitle("PCBs detection proportion", fontsize = 18, fontweight = "bold")
for i, col in enumerate(pcb_corrected.columns):
    broken = pcb_corrected.index[pcb_corrected.loc[:, col].isna() & pcb_corrected.index.str.contains("PCB-")]
    count = broken.to_series().apply(lambda x : x.count('+')).to_numpy().sum() + broken.size
    ax[i].pie([209-count, count], labels = ["", ""], autopct='%1.1f%%', startangle=90, radius = 1.4, colors = ['lightgray', 'limegreen'])
    ax[i].set_xlabel(f'#{col}', fontsize = 13)
fig.legend(["detected", "not detected"], loc = 'upper right', bbox_to_anchor=(.9, 1), fontsize = 13);
plt.tight_layout()

## WG85756_DX_1

In [ ]:
col_names = pd.read_excel("WG85756_DX_1.xls", sheet_name="DATA", nrows=0).columns
col_names = np.array(col_names[~col_names.str.contains("Unnamed")][1:].repeat(3))
for i in np.arange(tag_list.shape[0]):
    col_names[col_names == tag_list[i, 1]] = tag_list[i, 0][1:]
DX = pd.read_excel("WG85756_DX_1.xls", sheet_name="DATA", skiprows = 5)
DX = DX.loc[:int(np.where(DX['UNITS'] == r"% Lipid")[0]), :]
suffix = ['flag', 'value', 'RL'] * int(len(col_names) / 3)
cols = [i + '_' + j for i, j in zip(col_names, suffix)]
cols.insert(0, 'Units')
DX.columns = cols
DX.set_index('Units', inplace=True)

blank_columns = np.where(DX.columns.str.contains('Blank'))[0]
rl_nd = DX.loc[DX.iloc[:, blank_columns[0]].str.contains(r'\bND\b(?!R)', na = False).to_numpy(), DX.columns[blank_columns[2]]].dropna()
DX
# Unit: pg/g (wet weight basis)

### Blank correction

In [ ]:
DX_corrected = DX.copy().iloc[:, 1:-2:3]
DX_corrected.columns = DX_corrected.columns.str.removesuffix('_value')
DX_corrected.loc[rl_nd.index, DX_corrected.columns[-1]] = rl_nd.to_numpy() / np.sqrt(2)
id = DX_corrected.iloc[:-1, -1].dropna().index
DX_corrected.loc[id, DX_corrected.columns[:-1]] -= DX_corrected.loc[id, DX_corrected.columns[-1]].to_numpy()[:, np.newaxis]
DX_corrected = DX_corrected.iloc[:, :-1]
print(f'count of samples corrected to negative: {np.sum((~(DX_corrected.isna() | (DX_corrected > 0)).to_numpy()))}')
print(f'position of corrected samples: {np.where((~(DX_corrected.isna() | (DX_corrected > 0))).to_numpy())}')
DX_corrected.iloc[np.where((~(DX_corrected.isna() | (DX_corrected > 0))).to_numpy())] = 0


### Lipid normalization

In [379]:
lipid_normalized_dx = DX_corrected.copy()
lipid_normalized_dx.iloc[-1, :] /= 100
lipid_normalized_dx.iloc[:-1, :] /= lipid_normalized_dx.iloc[-1, :]
lipid_normalized_dx.index = list(lipid_normalized_dx.index[:-1])+['Lipid']


In [380]:
if not os.path.exists("./generated excels/"):
    os.mkdir("./generated excels/")
with pd.ExcelWriter('./generated excels/DX.xlsx') as writer:
    DX_corrected.to_excel(writer, sheet_name="blank_corrected")
    lipid_normalized_dx.to_excel(writer, sheet_name="lipid_normalized")

# DX graphs

In [381]:
DX_corrected = pd.read_excel("./generated excels/DX.xlsx", sheet_name="lipid_normalized")
dx = DX_corrected.T
dx.reset_index(inplace=True)
cols = dx.iloc[0, :].to_list()
cols[0] = 'Fish_ID'
dx.columns = cols
dx.drop(index = 0, inplace = True)
dx.reset_index(drop = True, inplace = True)
dx['Fish_ID'] = dx['Fish_ID'].str.removesuffix('-Trembleur')
dx.set_index('Fish_ID', inplace = True)

In [ ]:
dx

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), facecolor = "white")
(dx.loc[dx.sum(1) != 0, dx.columns[:-1]]/1000000).plot(use_index=True, kind = 'bar', stacked = True, title = 'Polychlorinated dibenzo-p-dioxins (PCDDs) and dibenzofurans (PCDFs)',
          xlabel = 'Fish ID', colormap= plt.colormaps["tab20c"], ax = ax);
plt.ylabel('mg/kg lw', fontsize = 15)
plt.xlabel('Fish ID', fontsize = 15)
plt.title('Polychlorinated dibenzo-p-dioxins (PCDDs) and dibenzofurans (PCDFs)', fontsize = 20)
plt.xticks(rotation = 0, fontweight = "bold", fontsize = 12)
plt.yticks(fontsize = 12)
plt.legend(bbox_to_anchor = (1, 1))
if not os.path.exists("./pics/DX"):
        os.mkdir("./pics/DX")
plt.savefig(f"./pics/DX/stacked_DX.png")

In [384]:
dx.drop(columns= dx.columns[dx.columns.str.contains("lipid", case = False)], inplace = True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), facecolor = "white")
(dx.T / (dx.sum(1) + 1e-20)*100).loc[:, dx.sum(1) != 0].T.plot(use_index=True, kind = 'bar', stacked = True, title = 'DX', ylabel = 'percent (%)', xlabel = 'Fish ID', colormap= plt.colormaps["tab20c"], ax = ax)
plt.ylabel('Percent (%)', fontsize = 15)
plt.xlabel('Fish ID', fontsize = 15)
plt.title('Polychlorinated dibenzo-p-dioxins (PCDDs) and dibenzofurans (PCDFs)', fontsize = 20)
plt.xticks(rotation = 0, fontweight = "bold", fontsize = 12)
plt.yticks(fontsize = 12)
plt.legend(bbox_to_anchor=(1.0, 1));
if not os.path.exists("./pics/DX"):
        os.mkdir("./pics/DX")
plt.savefig(f"./pics/DX/stacked_percent_DX.png")

# PCB graphs

In [386]:
pcb_corrected = pd.read_excel("./generated excels/pcb.xlsx", sheet_name="blank_corrected")

In [387]:
pcb_corrected = pcb_corrected.iloc[:-12, :]
pcb_corrected.set_index('Units', inplace=True)

In [388]:
import math as m

In [ ]:
xtick_labels = []
xticks = list(np.arange(0, pcb_corrected.shape[0], 15)) + [pcb_corrected.shape[0]-1]
for i in xticks:
    xtick_labels.append(pcb_corrected.index[i])
for col in pcb_corrected:
    fig, ax = plt.subplots(figsize = (15, 10))
    ax.set_title(col)
    ax.bar(pcb_corrected.index, pcb_corrected[col], color = 'k')
    ax.xaxis.set_ticks(xticks)
    ax.xaxis.set_ticklabels(xtick_labels, rotation = 40, ha = "right")
    ax.set_ylabel("pg/g (wet weight basis)")
    ax.set_xlabel("$PCB_x$")
    for iter, i in enumerate(pcb_corrected.index):
        if '+' in i:
            ax.annotate(i,
            xy=(iter, pcb_corrected.loc[i, col]), xycoords='data',
            xytext=(iter+10, pcb_corrected.loc[i, col]+50), textcoords='data',
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3"), rotation = 73)


In [ ]:
xtick_labels = []
xticks = list(np.arange(0, pcb_corrected.shape[0], 15)) + [pcb_corrected.shape[0]-1]
for i in xticks:
    xtick_labels.append(pcb_corrected.index[i])
for col in pcb_corrected:
    fig, ax = plt.subplots(figsize = (15, 10))
    ax.set_title(col)
    ax.bar(pcb_corrected.index, pcb_corrected[col], color = 'k')
    ax.xaxis.set_ticks(xticks)
    ax.xaxis.set_ticklabels(xtick_labels, rotation = 40, ha = "right")
    ax.set_ylabel("pg/g (wet weight basis)")
    ax.set_xlabel("$PCB_x$")

### all compounds are muted

In [ ]:
id = ~pcb_corrected.index.str.contains("+", regex=False)
xtick_labels = []
xticks = list(np.arange(0, pcb_corrected.loc[id, :].shape[0], 15)) + [pcb_corrected.loc[id, :].shape[0]-1]
for i in xticks:
    xtick_labels.append(pcb_corrected.loc[id, :].index[i])
for col in pcb_corrected:
    fig, ax = plt.subplots(figsize = (15, 10))
    ax.set_title(col)
    ax.bar(pcb_corrected.index[id], pcb_corrected.loc[id, col], color = 'k')
    ax.xaxis.set_ticks(xticks)
    ax.xaxis.set_ticklabels(xtick_labels, rotation = 40, ha = "right")
    ax.set_ylabel("pg/g (wet weight basis)")
    ax.set_xlabel("$PCB_x$")

### splited compounds

In [ ]:
for col in pcb_corrected:
    pcbx = np.zeros((209,))
    id = ~pcb_corrected.index.str.contains("+", regex=False)
    idx = pcb_corrected[id].index.str.removeprefix("PCB-").to_numpy('uint8')-1
    pcbx[idx] = pcb_corrected.loc[id, col].ravel()
    for i in pcb_corrected[~id].index:
        for k , j in enumerate(i.split('+')):
            if k == 0:
                pcbx[int(j.strip()[4:])-1] = pcb_corrected.loc[i, col] / len(i.split('+'))
            else:
                pcbx[int(j.strip())-1] = pcb_corrected.loc[i, col] / len(i.split('+'))
    fig, ax = plt.subplots(figsize = (15, 10))
    ax.set_title(col)
    ax.bar(np.arange(1, 210), pcbx, color = 'k')
    ax.xaxis.set_ticks(np.arange(1, 210, 10))
    ax.xaxis.set_ticklabels(np.arange(1, 210, 10), rotation = 40, ha = "right")
    ax.set_ylabel("pg/g (wet weight basis)")
    ax.set_xlabel("$PCB_x$")


# age table

In [ ]:
age_table = pd.read_excel('./2022 Mortality Fin Ray Aging.xlsx')
age_table['Sample'] = age_table['Sample'].str.removeprefix("WS")
age_table

In [ ]:
pcb_corrected

# Ploting pcb vs age

In [ ]:
ages = [age_table.loc[age_table["Sample"].str.contains(c[:2]), 'Age Estimate'].to_numpy()[0] for c in pcb_corrected.columns]
for pcb_i in pcb_corrected.index:
    if ~np.isnan(pcb_corrected.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(ages, pcb_corrected.loc[pcb_i, :].to_numpy())
        plt.xlabel("Age")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = pcb_corrected.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            slope, intercept, r_value, p_value, std_err = linregress(np.array(ages, dtype=np.single)[temp].ravel(), pcb_corrected.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            model = LinearRegression()
            model.fit(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), pcb_corrected.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = r_value ** 2
            plt.plot(ages, model.predict(np.array(ages).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\npcb = {model.coef_[0][0]:.2f} * Age + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/pcbs/scatter/Age"):
        os.mkdir("./pics/pcbs/scatter")
        os.mkdir("./pics/pcbs/scatter/Age")
    plt.savefig(f"./pics/pcbs/scatter/Age/{pcb_i}.png")

# DX vs Age

In [396]:
dx = dx.T

In [ ]:
ages = [age_table.loc[age_table["Sample"].str.contains(c[:2]), 'Age Estimate'].to_numpy()[0] for c in dx.columns]
for dx_i in dx.index:
    if ~np.isnan(dx.loc[dx_i, :].to_numpy('single')).all():
        plt.figure(figsize=(10, 10), facecolor = 'White')
        plt.scatter(ages, dx.loc[dx_i, :].to_numpy('double'))
        plt.xlabel("Age")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{dx_i}')
        temp = dx.loc[dx_i, :].to_numpy() > 0
        if temp.sum()>1:
            slope, intercept, r_value, p_value, std_err = linregress(np.array(ages, dtype=np.single)[temp].ravel(), dx.loc[dx_i, :].to_numpy('single')[temp].ravel())
            model = LinearRegression()
            model.fit(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), dx.loc[dx_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), dx.loc[dx_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(ages, model.predict(np.array(ages).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\nDx = {model.coef_[0][0]:.2f} * Age + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                        (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/Dx/scatter/Age"):
        os.mkdir("./pics/Dx/scatter")
        os.mkdir("./pics/Dx/scatter/Age")
    plt.savefig(f"./pics/Dx/scatter/Age/{dx_i}.png")

# pcb vs weight and fulton factor

In [ ]:
fish_info = fish_dict['Aluminum'][0].loc[:, ['tag', 'weight', 'fulton']]
fish_info['tag'].iloc[-1] = '13'
fish_info

In [ ]:
weight = fish_info['weight'].to_numpy('single')
pcb_corrected.rename(columns={'13-Trembleur': '13'}, inplace = True)
p = pcb_corrected.loc[:, fish_info['tag'].to_numpy('str')]
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor="white")
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy())
        plt.xlabel("Weight (Kg)")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\npcb = {model.coef_[0][0]:.2f} * weight + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/pcbs/scatter/Weight"):
        os.mkdir("./pics/pcbs/scatter/Weight")
    plt.savefig(f"./pics/pcbs/scatter/Weight/{pcb_i}.png")

In [ ]:
weight = fish_info['fulton'].to_numpy('single')
p = pcb_corrected.loc[:, fish_info['tag'].to_numpy('str')]
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor="white")
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy())
        plt.xlabel("Fulton's condition factor")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\npcb = {model.coef_[0][0]:.2f} * fulton + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/pcbs/scatter/fulton"):
        os.mkdir("./pics/pcbs/scatter/fulton")
    plt.savefig(f"./pics/pcbs/scatter/fulton/{pcb_i}.png")

# DX vs weight and fulton

In [ ]:
dx

In [ ]:
weight = fish_info['weight'].to_numpy('single')
p = dx.loc[:, fish_info['tag'].to_numpy('str')]
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy('single')).all():
        plt.figure(figsize=(10, 10), facecolor = 'White')
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy('double'))
        plt.xlabel("Weight (Kg)")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy('single') > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\nDx = {model.coef_[0][0]:.2f} * weight + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/Dx/scatter/Weight"):
        os.mkdir("./pics/Dx/scatter/Weight")
    plt.savefig(f"./pics/Dx/scatter/Weight/{pcb_i}.png")

In [ ]:
weight = fish_info['fulton'].to_numpy('single')
p = dx.loc[:, fish_info['tag'].to_numpy('str')]
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy('single')).all():
        plt.figure(figsize=(10, 10), facecolor = 'White')
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy('double'))
        plt.xlabel("Fulton's condition factor")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy('single') > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\nDx = {model.coef_[0][0]:.2f} * fulton + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/Dx/scatter/fulton"):
        os.mkdir("./pics/Dx/scatter/fulton")
    plt.savefig(f"./pics/Dx/scatter/fulton/{pcb_i}.png")

# Repeating all with Log

In [ ]:
ages = [age_table.loc[age_table["Sample"].str.contains(c[:2]), 'Age Estimate'].to_numpy()[0] for c in pcb_corrected.columns]
for pcb_i in pcb_corrected.index:
    if ~np.isnan(pcb_corrected.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(ages, np.log10(pcb_corrected.loc[pcb_i, :].to_numpy()))
        plt.xlabel("Age")
        plt.ylabel("log10 pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = pcb_corrected.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            slope, intercept, r_value, p_value, std_err = linregress(np.array(ages, dtype=np.single)[temp].ravel(), np.log10(pcb_corrected.loc[pcb_i, :].to_numpy('single')[temp].ravel()))
            model = LinearRegression()
            history = model.fit(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), np.log10(pcb_corrected.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1)))
            r_sq = r_value ** 2
            plt.plot(ages, model.predict(np.array(ages).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'$log_{10}^{(pcb)}$' + f"= {model.coef_[0][0]:.2f} * Age + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/pcbs/scatter/Age_log"):
        os.mkdir("./pics/pcbs/scatter/Age_log")
    plt.savefig(f"./pics/pcbs/scatter/Age_log/{pcb_i}.png")

In [ ]:
ages = [age_table.loc[age_table["Sample"].str.contains(c[:2]), 'Age Estimate'].to_numpy()[0] for c in dx.columns]
for dx_i in dx.index:
    if ~np.isnan(dx.loc[dx_i, :].to_numpy('single')).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(ages, np.log10(dx.loc[dx_i, :].to_numpy('double')))
        plt.xlabel("Age")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{dx_i}')
        temp = dx.loc[dx_i, :].to_numpy() > 0
        if temp.sum()>1:
            slope, intercept, r_value, p_value, std_err = linregress(np.array(ages, dtype=np.single)[temp].ravel(), np.log10(dx.loc[dx_i, :].to_numpy('single')[temp].ravel()))
            model = LinearRegression()
            model.fit(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), np.log10(dx.loc[dx_i, :].to_numpy('single')[temp].reshape(-1, 1)))
            r_sq = model.score(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), np.log10(dx.loc[dx_i, :].to_numpy('single')[temp].reshape(-1, 1)))
            plt.plot(ages, model.predict(np.array(ages).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n"+r'$log_{10}^{Dx}' + f" = {model.coef_[0][0]:.2f} * Age + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                        (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/Dx/scatter/Age_log"):
        os.mkdir("./pics/Dx/scatter/Age_log")
    plt.savefig(f"./pics/Dx/scatter/Age_log/{dx_i}.png")

In [ ]:
weight = fish_info['weight'].to_numpy('single')
p = np.log10(pcb_corrected.loc[:, fish_info['tag'].to_numpy('str')])
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor="white")
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy())
        plt.xlabel("Weight (Kg)")
        plt.ylabel("log10 pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'$log_{10}^{pcb}$' f" = {model.coef_[0][0]:.2f} * weight + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/pcbs/scatter/Weight_log"):
        os.mkdir("./pics/pcbs/scatter/Weight_log")
    plt.savefig(f"./pics/pcbs/scatter/Weight_log/{pcb_i}.png")

In [ ]:
weight = fish_info['fulton'].to_numpy('single')
p = np.log10(pcb_corrected.loc[:, fish_info['tag'].to_numpy('str')])
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor="white")
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy())
        plt.xlabel("Fulton's condition factor")
        plt.ylabel("log10 pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'$log_{10}^{pcb}$' f" = {model.coef_[0][0]:.2f} * fulton + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/pcbs/scatter/fulton_log"):
        os.mkdir("./pics/pcbs/scatter/fulton_log")
    plt.savefig(f"./pics/pcbs/scatter/fulton_log/{pcb_i}.png")

In [ ]:
weight = fish_info['weight'].to_numpy('single')
p = np.log10(dx.loc[:, fish_info['tag'].to_numpy('str')].astype("double"))
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy('single')).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy('double'))
        plt.xlabel("Weight (Kg)")
        plt.ylabel("log10 pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy('single') > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'$log_{10}^{Dx}$' f" = {model.coef_[0][0]:.2f} * weight + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/Dx/scatter/Weight_log"):
        os.mkdir("./pics/Dx/scatter/Weight_log")
    plt.savefig(f"./pics/Dx/scatter/Weight_log/{pcb_i}.png")

In [ ]:
weight = fish_info['fulton'].to_numpy('single')
p = np.log10(dx.loc[:, fish_info['tag'].to_numpy('str')].astype("double"))
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy('single')).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy('double'))
        plt.xlabel("Fulton's condition factor")
        plt.ylabel("log10 pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy('single') > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'$log_{10}^{Dx}$' f" = {model.coef_[0][0]:.2f} * fulton + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0],0.01*np.diff(plt.ylim())[0]+plt.ylim()[0]), color = 'k', fontweight = "bold")
    if not os.path.exists("./pics/Dx/scatter/fulton_log"):
        os.mkdir("./pics/Dx/scatter/fulton_log")
    plt.savefig(f"./pics/Dx/scatter/fulton_log/{pcb_i}.png")

# Repeating all for lipid normalized

In [410]:
pcb_lipid = pd.read_excel("./generated excels/pcb.xlsx", sheet_name="lipid_normalized")
pcb_lipid = pcb_lipid.iloc[:-12, :]
c = pcb_lipid.columns.tolist()
c[0] = "Units"
pcb_lipid.columns = c
pcb_lipid.set_index('Units', inplace=True)

In [ ]:
pcb_lipid

In [ ]:
ages = [age_table.loc[age_table["Sample"].str.contains(c[:2]), 'Age Estimate'].to_numpy()[0] for c in pcb_lipid.columns]
for pcb_i in pcb_lipid.index:
    if ~np.isnan(pcb_lipid.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(ages, pcb_lipid.loc[pcb_i, :].to_numpy())
        plt.xlabel("Age")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = pcb_lipid.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            slope, intercept, r_value, p_value, std_err = linregress(np.array(ages, dtype=np.single)[temp].ravel(), pcb_lipid.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            model = LinearRegression()
            history = model.fit(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), pcb_lipid.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = r_value ** 2
            plt.plot(ages, model.predict(np.array(ages).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + 'pcb' + f"= {model.coef_[0][0]:.2f} * Age + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0], .1 * np.diff(plt.ylim())[0]+plt.ylim()[1]), color = 'k', fontweight = "bold")
            plt.ylim(plt.ylim()[0], .3 * np.diff(plt.ylim())[0]+plt.ylim()[1])
    if not os.path.exists("./pics/pcbs/scatter/Age_lipid"):
        os.mkdir("./pics/pcbs/scatter/Age_lipid")
    plt.savefig(f"./pics/pcbs/scatter/Age_lipid/{pcb_i}.png")

In [413]:
dx = pd.read_excel("./generated excels/DX.xlsx", sheet_name = "lipid_normalized")
dx = dx.iloc[:-9, :]
dx.columns = c
dx.set_index('Units', inplace=True)

In [ ]:
ages = [age_table.loc[age_table["Sample"].str.contains(c[:2]), 'Age Estimate'].to_numpy()[0] for c in dx.columns]
for dx_i in dx.index:
    if ~np.isnan(dx.loc[dx_i, :].to_numpy('single')).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(ages, dx.loc[dx_i, :].to_numpy('double'))
        plt.xlabel("Age")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{dx_i}')
        temp = dx.loc[dx_i, :].to_numpy() > 0
        if temp.sum()>1:
            slope, intercept, r_value, p_value, std_err = linregress(np.array(ages, dtype=np.single)[temp].ravel(), dx.loc[dx_i, :].to_numpy('single')[temp].ravel())
            model = LinearRegression()
            model.fit(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), dx.loc[dx_i, :].to_numpy('single')[temp].reshape(-1, 1))
            r_sq = model.score(np.array(ages, dtype=np.single)[temp].reshape(-1, 1), dx.loc[dx_i, :].to_numpy('single')[temp].reshape(-1, 1))
            plt.plot(ages, model.predict(np.array(ages).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n"+ r'Dx' + f" = {model.coef_[0][0]:.2f} * Age + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                        (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0], .1 * np.diff(plt.ylim())[0]+plt.ylim()[1]), color = 'k', fontweight = "bold")
            plt.ylim(plt.ylim()[0], .3 * np.diff(plt.ylim())[0]+plt.ylim()[1])
    if not os.path.exists("./pics/Dx/scatter/Age_lipid"):
        os.mkdir("./pics/Dx/scatter/Age_lipid")
    plt.savefig(f"./pics/Dx/scatter/Age_lipid/{dx_i}.png")

In [ ]:
weight = fish_info['weight'].to_numpy('single')
pcb_lipid.rename(columns={'13-Trembleur': '13'}, inplace = True)
p = pcb_lipid.loc[:, fish_info['tag'].to_numpy('str')]
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor="white")
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy())
        plt.xlabel("Weight (Kg)")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'$pcb$' f" = {model.coef_[0][0]:.2f} * weight + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0], .1 * np.diff(plt.ylim())[0]+plt.ylim()[1]), color = 'k', fontweight = "bold")
            plt.ylim(plt.ylim()[0], .3 * np.diff(plt.ylim())[0]+plt.ylim()[1])
    if not os.path.exists("./pics/pcbs/scatter/Weight_lipid"):
        os.mkdir("./pics/pcbs/scatter/Weight_lipid")
    plt.savefig(f"./pics/pcbs/scatter/Weight_lipid/{pcb_i}.png")

In [ ]:
weight = fish_info['fulton'].to_numpy('single')
p = pcb_corrected.loc[:, fish_info['tag'].to_numpy('str')]
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor="white")
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy())
        plt.xlabel("Fulton's condition factor")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'pcb' f" = {model.coef_[0][0]:.2f} * fulton + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0], .1 * np.diff(plt.ylim())[0]+plt.ylim()[1]), color = 'k', fontweight = "bold")
            plt.ylim(plt.ylim()[0], .3 * np.diff(plt.ylim())[0]+plt.ylim()[1])
    if not os.path.exists("./pics/pcbs/scatter/fulton_lipid"):
        os.mkdir("./pics/pcbs/scatter/fulton_lipid")
    plt.savefig(f"./pics/pcbs/scatter/fulton_lipid/{pcb_i}.png")

In [ ]:
weight = fish_info['fulton'].to_numpy('single')
p = pcb_corrected.loc[:, fish_info['tag'].to_numpy('str')]
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy()).all():
        plt.figure(figsize=(6, 4), facecolor="white")
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy())
        plt.xlabel("Fulton's condition factor")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy() > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'pcb' f" = {model.coef_[0][0]:.2f} * fulton + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0], .1 * np.diff(plt.ylim())[0]+plt.ylim()[1]), color = 'k', fontweight = "bold")
            plt.ylim(plt.ylim()[0], .3 * np.diff(plt.ylim())[0]+plt.ylim()[1])
    if not os.path.exists("./pics/pcbs/scatter/fulton_lipid"):
        os.mkdir("./pics/pcbs/scatter/fulton_lipid")
    plt.savefig(f"./pics/pcbs/scatter/fulton_lipid/{pcb_i}.png")

In [ ]:
weight = fish_info['fulton'].to_numpy('single')
dx.rename(columns={'13-Trembleur': '13'}, inplace = True)
p = dx.loc[:, fish_info['tag'].to_numpy('str')].astype("double")
for pcb_i in p.index:
    if ~np.isnan(p.loc[pcb_i, :].to_numpy('single')).all():
        plt.figure(figsize=(6, 4), facecolor = 'White')
        plt.scatter(weight, p.loc[pcb_i, :].to_numpy('double'))
        plt.xlabel("Fulton's condition factor")
        plt.ylabel("pg/g (wet weight basis)")
        plt.title(f'{pcb_i}')
        temp = p.loc[pcb_i, :].to_numpy('single') > 0
        if temp.sum() > 1:
            model = LinearRegression()
            slope, intercept, r_value, p_value, std_err = linregress(weight[temp], p.loc[pcb_i, :].to_numpy('single')[temp].ravel())
            history = model.fit(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            r_sq = model.score(np.array(weight, dtype=np.single)[temp].reshape(-1, 1), p.loc[pcb_i, :].to_numpy()[temp].reshape(-1, 1))
            plt.plot(weight, model.predict(np.array(weight).reshape(-1, 1)), color = 'orange')
            plt.annotate(f"$r^2$ = {r_sq:.4f}\n" + r'$Dx' f" = {model.coef_[0][0]:.2f} * fulton + {model.intercept_[0]:.2f}\nP-Value = {p_value:.2f}",
                      (0.01*np.diff(plt.xlim())[0]+plt.xlim()[0], .1 * np.diff(plt.ylim())[0]+plt.ylim()[1]), color = 'k', fontweight = "bold")
            plt.ylim(plt.ylim()[0], .3 * np.diff(plt.ylim())[0]+plt.ylim()[1])
    if not os.path.exists("./pics/Dx/scatter/fulton_lipid"):
        os.mkdir("./pics/Dx/scatter/fulton_lipid")
    plt.savefig(f"./pics/Dx/scatter/fulton_lipid/{pcb_i}.png")

# Multi-Panel Box Plot

In [ ]:
analytes = punch_data.Analyte.unique()
analytes = analytes[[0, 2, 1, -2, 3, -1]]
fig, axs = plt.subplots(3, 2, facecolor = 'white', figsize = (12, 15), sharey = 'row')
axs = axs.flatten()
for i, analyte in enumerate(analytes):
    # plt.subplot(3, 2, i+1)
    result = punch_data.loc[(punch_data.Matrix == 'Fish') & (punch_data.Analyte == analyte) & ~punch_data.Sample_Tag.str.contains('13'), ['Tissue', 'Sample_Tag', 'Ours', 'Result']]
    id = result.Tissue == 'Liver'
    plot_data = np.c_[result.loc[id, ['Ours']].to_numpy(), result.loc[~id, ['Ours', 'Result']].to_numpy()]
    plot_data = plot_data.T.tolist()
    if np.isnan(plot_data[0][-1]):
        del plot_data[0][-1]
    axs[i].boxplot(plot_data);
    axs[i].set_title(analyte, fontweight = 'bold', fontsize = 14)
    axs[2].set_ylabel('mg/kg wwt.', fontweight = 'bold', fontsize = 14)
    if i < 4: 
        axs[i].tick_params(labelbottom=False)
    else:
        axs[i].set_xticklabels(['Liver', 'Muscle', 'Punch'], rotation = 45, fontsize = 14, fontweight = 'bold')


## Quartiles

In [457]:
t = data
quartiles = t.drop(columns=t.columns[3:])
quartiles['tag'] = 'MT'
qs = np.quantile(t.iloc[:, 3:12].to_numpy('float32'), [0, .25, .5, .75, 1], axis = 1)
qs = pd.DataFrame(qs.T, columns=['Q0', 'Q1', 'Q2', 'Q3', 'Q4'])
quartiles = pd.concat([quartiles, qs], axis = 1)
qs = quartiles.copy()
qs.tag = 'Liver'
qs.iloc[:, 4:] = np.quantile(t.iloc[:, 12:].to_numpy('float32'), [0, .25, .5, .75, 1], axis = 1).T
quartiles = pd.concat([quartiles, qs], axis = 0)
quartiles = quartiles.sort_values('Analyte').reset_index(drop = True)
quartiles.to_excel('quartiles.xlsx')